In [3]:
import numpy as np

def kl_loss(p, q):
 return sum(p[i] * np.log(p[i]/q[i]) for i in range(len(p)))

p = [0.10, 0.40, 0.50]
q = [0.80, 0.15, 0.05]
kl_loss(p, q)

1.3356800935337296

### Binary Cross-entropy

In [5]:
import torch.nn as nn
import torch

def sigmoid(x):
    return 1/(1+np.exp(-x))
# Binary Cross Entropy loss for multi-label problem => use sigmoid.
def binary_cross_entropy(y_pred, y):
    y_pred = sigmoid(y_pred)
    loss = -1* np.sum(y * np.log(y_pred) + (1-y) * np.log(1-y_pred))
    return loss

y_pred = np.array([1, 0, 0, 0, 0])
y = np.array([0, 1, 1, 1, 0])
binary_cross_entropy(y_pred, y)

4.085850409758004

## Cross Entropy

In [8]:
# Cross Entropy function.
import numpy as np
# Defining the softmax function
def softmax(values):
    exp_values = np.exp(values)
    exp_values_sum = np.sum(exp_values)
    return exp_values/exp_values_sum

def cross_entropy_loss(y_pred, y):
        y_pred = softmax(y_pred)
        correct_logprobs = -np.log(y_pred[range(len(y_pred)), y])
        loss = np.sum(correct_logprobs)
        return 1.0/len(y_pred) * loss

## Focal Loss 

Focal Loss is particularly useful in cases where there is a class imbalance. What Focal Loss does is that it makes it easier for the model to predict things without being 80-100% sure that this object is “something”. 
([refer](https://amaarora.github.io/posts/2020-06-29-FocalLoss.html))


![image.png](../imgs/focal.png)

In [14]:
def focal_loss(y_true, y_pred, alpha=0.25, gamma=2):
    """Focal loss is used to address the issue of the class imbalance problem. A modulation term applied to the Cross-Entropy loss function.
    Parameters
    ----------
    alpha : float, optional
        controls relative weight of false positives and false negatives. alpha > 0.5 penalises false negatives more than false positives, by default None
    gamma_f : float, optional
        focal parameter controls degree of down-weighting of easy examples, by default 2.
    """
    epsilon = 1e-9  # Clip values to prevent division by zero error
    y_pred = np.clip(y_pred, epsilon, 1.0 - epsilon)  # Clip to prevent numerical instability
    # Calculate cross entropy
    cross_entropy = -y_true * np.log(y_pred)
    # Calculate focal weights
    focal_weights = alpha * np.power(1 - y_pred, gamma)
    # Combine focal weights with cross entropy
    focal_loss = focal_weights * cross_entropy
    # Sum along classes to get loss for each sample
    focal_loss = np.sum(focal_loss, axis=-1)
    return focal_loss

y_true = np.array([[1, 0, 0], [0,1,0]])
y_pred = np.array([[0.9, 0.1, 0], [0.4,0.3,0.3]])
focal_loss(y_true, y_pred)


array([0.0002634 , 0.14748667])

## CLIP loss

In [2]:
# Calculating the Loss
def clip_loss(text_embeddings, image_embeddings):
    logits = (text_embeddings @ image_embeddings.T) / self.temperature
    images_similarity = image_embeddings @ image_embeddings.T
    texts_similarity = text_embeddings @ text_embeddings.T
    targets = F.softmax(
        (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
    )
    texts_loss = cross_entropy(logits, targets, reduction='none')
    images_loss = cross_entropy(logits.T, targets.T, reduction='none')
    loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
    return loss.mean()